# Labs Access Analysis: 01 Data Download

# Setup

In [1]:
import pandas as pd
import numpy as np
import requests
from openrouteservice import client
import time
import json

In [2]:
# Load data
labs = pd.read_csv('data/labs.csv')

In [3]:
labs

,Name,Type,City,Cluster,Address,CAP,Lat,Long,Source,URL,Notes
0,Fab Lab Barcelona,Fab Lab,Barcelona,BCN,"Carrer de Pujades, 102",8005.0,41.396925,2.194335,Fablabs.io,https://fablabbcn.org/,NaN
1,MADE Makerspace Barcelona,Fab Lab,Barcelona,BCN,Carrer Noguera Pallaresa 59-61,8014.0,41.373238,2.141084,"Fablabs.io, Hackerspaces.org, Makerspaces.make.co",NaN,NaN
2,Soko Tech,Fab Lab,Barcelona,BCN,"Carrer de Vallès i Ribot, 36",8027.0,41.424535,2.188883,Fablabs.io,https://soko.tech/,NaN
3,LSCongres,Fab Lab,Barcelona,BCN,"Carrer del Cardenal Tedeschini, 50",8027.0,41.426002,2.182026,Fablabs.io,https://fablabs.io/labs/lscongres,NaN
4,Green Fab Lab,Fab Lab,Cerdanyola del Vallès,BCN,"Ctra. BV-1415 (Horta-Cerdanyola), km 7",8290.0,41.450394,2.133441,Fablabs.io,NaN,NaN
5,Barcelona Mobility LAB,Fab Lab,Barcelona,BCN,Passatge del Torrent de l'Estadella 56,8030.0,41.432152,2.199793,Fablabs.io,NaN,NaN
6,Fab Lab Sant Cugat,Fab Lab,Sant Cugat del Vallès,BCN,"Avinguda de la Torre Blanca, 57",8712.0,41.467569,2.091704,Fablabs.io,NaN,NaN
7,Frankestein Garage,Fab Lab,Milan,MI,NaN,NaN,NaN,NaN,"Fablabs.io, Hackerspaces.org",https://fablabs.io/labs/frankensteingarage,Closed
8,The FabLab: Make in Milano,Fab Lab,Milan,MI,"Via Arcivescovo Calabiana, 6",20139.0,45.445866,9.208566,Fablabs.io,NaN,NaN
9,Vectorealism FabLab,Service,Milan,MI,"Via Carlo Boncompagni, 57",20139.0,45.438305,9.231811,Fablabs.io,https://fablabs.io/labs/vectorealismfablab,NaN


In [4]:
closed_labs = labs[labs.Notes.str.contains("losed")==True]

In [5]:
valid_labs = labs[~labs.index.isin(closed_labs.index)]

In [6]:
valid_labs

,Name,Type,City,Cluster,Address,CAP,Lat,Long,Source,URL,Notes
0,Fab Lab Barcelona,Fab Lab,Barcelona,BCN,"Carrer de Pujades, 102",8005.0,41.396925,2.194335,Fablabs.io,https://fablabbcn.org/,NaN
1,MADE Makerspace Barcelona,Fab Lab,Barcelona,BCN,Carrer Noguera Pallaresa 59-61,8014.0,41.373238,2.141084,"Fablabs.io, Hackerspaces.org, Makerspaces.make.co",NaN,NaN
2,Soko Tech,Fab Lab,Barcelona,BCN,"Carrer de Vallès i Ribot, 36",8027.0,41.424535,2.188883,Fablabs.io,https://soko.tech/,NaN
3,LSCongres,Fab Lab,Barcelona,BCN,"Carrer del Cardenal Tedeschini, 50",8027.0,41.426002,2.182026,Fablabs.io,https://fablabs.io/labs/lscongres,NaN
4,Green Fab Lab,Fab Lab,Cerdanyola del Vallès,BCN,"Ctra. BV-1415 (Horta-Cerdanyola), km 7",8290.0,41.450394,2.133441,Fablabs.io,NaN,NaN
5,Barcelona Mobility LAB,Fab Lab,Barcelona,BCN,Passatge del Torrent de l'Estadella 56,8030.0,41.432152,2.199793,Fablabs.io,NaN,NaN
6,Fab Lab Sant Cugat,Fab Lab,Sant Cugat del Vallès,BCN,"Avinguda de la Torre Blanca, 57",8712.0,41.467569,2.091704,Fablabs.io,NaN,NaN
8,The FabLab: Make in Milano,Fab Lab,Milan,MI,"Via Arcivescovo Calabiana, 6",20139.0,45.445866,9.208566,Fablabs.io,NaN,NaN
9,Vectorealism FabLab,Service,Milan,MI,"Via Carlo Boncompagni, 57",20139.0,45.438305,9.231811,Fablabs.io,https://fablabs.io/labs/vectorealismfablab,NaN
10,DamASpace,Fab Lab,Milan,MI,"Corso S. Gottardo, 19",20136.0,45.452457,9.179818,Fablabs.io,https://fablabs.io/labs/DamASpaceFablab,NaN


In [7]:
valid_labs['Lat'] = valid_labs['Lat'].astype(float)
valid_labs['Long'] = valid_labs['Long'].astype(float)

/var/folders/77/s06kbcv90dzbmlh4swml9sdw0000gn/T/ipykernel_58476/3211071250.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_labs['Lat'] = valid_labs['Lat'].astype(float)
/var/folders/77/s06kbcv90dzbmlh4swml9sdw0000gn/T/ipykernel_58476/3211071250.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_labs['Long'] = valid_labs['Long'].astype(float)


In [8]:
# 900 seconds = 15 minutes (city)
body = {"locations":[],"range":[300,300],"attributes":["total_pop","area","reachfactor"],"range_type":"time","interval":900,"area_units":"m"}

locations = {}

In [9]:
for index, row in valid_labs.iterrows():
    if np.isnan(row["Lat"]) == False and np.isnan(row["Long"]) == False:
      locations[row["Name"]] = [row["Long"], row["Lat"]]

In [10]:
print(locations)

{'Fab Lab Barcelona': [2.194335155490849, 41.39692498100806], 'MADE Makerspace Barcelona': [2.141084282475449, 41.373238104403214], 'Soko Tech': [2.1888825978201867, 41.42453540945304], 'LSCongres': [2.1820259689844854, 41.426002192952815], 'Green Fab Lab': [2.133441411313727, 41.45039383574725], 'Barcelona Mobility LAB': [2.199793097820458, 41.43215238774583], 'Fab Lab Sant Cugat': [2.0917035978216068, 41.46756916570972], 'The FabLab: Make in Milano': [9.208566494559507, 45.44586623217766], 'Vectorealism FabLab': [9.231811340285022, 45.43830458381551], 'DamASpace': [9.1798175115845, 45.45245728345691], 'Miocugino - officina di gesta e ingegno': [9.16895578261383, 45.44821912386584], 'Opendot': [9.22096801664852, 45.45034823105788], 'Polifactory': [9.166295669123226, 45.506449418184594], 'WeMake | Makerspace Fablab': [9.1751237151523, 45.499039020857865], 'FabLab Ventura': [9.245129470972836, 45.4842083864999], 'FabLabMilano - Fare le Idee': [9.152947547090642, 45.50542134145349], 'Mul

In [15]:
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    # Get the authorization API key here: https://openrouteservice.org/
    'Authorization': '',
    'Content-Type': 'application/json; charset=utf-8'
}

In [12]:
body["locations"] = []
data = []
for k,i in enumerate(locations):
    print(i, locations[i])
    body["locations"].append(locations[i])
    # 15 minutes city, at walking distance
    call = requests.post('https://api.openrouteservice.org/v2/isochrones/foot-walking', json=body, headers=headers)
    body["locations"] = []
    print(call.status_code, call.reason)
    lab_dict = call.json()
    lab_dict["name"] = i
    data.append(lab_dict)
    print("---")
    time.sleep(30)

Fab Lab Barcelona [2.194335155490849, 41.39692498100806]
200 OK
---
MADE Makerspace Barcelona [2.141084282475449, 41.373238104403214]
200 OK
---
Soko Tech [2.1888825978201867, 41.42453540945304]
200 OK
---
LSCongres [2.1820259689844854, 41.426002192952815]
200 OK
---
Green Fab Lab [2.133441411313727, 41.45039383574725]
200 OK
---
Barcelona Mobility LAB [2.199793097820458, 41.43215238774583]
200 OK
---
Fab Lab Sant Cugat [2.0917035978216068, 41.46756916570972]
200 OK
---
The FabLab: Make in Milano [9.208566494559507, 45.44586623217766]
200 OK
---
Vectorealism FabLab [9.231811340285022, 45.43830458381551]
200 OK
---
DamASpace [9.1798175115845, 45.45245728345691]
200 OK
---
Miocugino - officina di gesta e ingegno [9.16895578261383, 45.44821912386584]
200 OK
---
Opendot [9.22096801664852, 45.45034823105788]
200 OK
---
Polifactory [9.166295669123226, 45.506449418184594]
200 OK
---
WeMake | Makerspace Fablab [9.1751237151523, 45.499039020857865]
200 OK
---
FabLab Ventura [9.245129470972836, 

In [13]:
data

[{'type': 'FeatureCollection',
  'bbox': [2.190392, 41.393927, 2.198272, 41.400117],
  'features': [{'type': 'Feature',
    'properties': {'group_index': 0,
     'value': 300.0,
     'center': [2.1943253081862886, 41.396932544923246],
     'area': 406405.17,
     'reachfactor': 0.7451,
     'total_pop': 10677.0},
    'geometry': {'coordinates': [[[2.190392, 41.397674],
       [2.190397, 41.397253],
       [2.190399, 41.397139],
       [2.190457, 41.39527],
       [2.19069, 41.394489],
       [2.190905, 41.394201],
       [2.191965, 41.394032],
       [2.194153, 41.393936],
       [2.194358, 41.39393],
       [2.194648, 41.393927],
       [2.196779, 41.394008],
       [2.19694, 41.394076],
       [2.197816, 41.394886],
       [2.198032, 41.395174],
       [2.1982, 41.396396],
       [2.198272, 41.397785],
       [2.198043, 41.399347],
       [2.197998, 41.399504],
       [2.197796, 41.399714],
       [2.19529, 41.400117],
       [2.193935, 41.400005],
       [2.191984, 41.399864],
     

In [14]:
with open("data/isochrones.json", 'w') as outfile:
    json.dump(data, outfile)